In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
from datetime import datetime
from pytz import timezone

/Users/shelbyjouppi/.pyenv/versions/3.9.7/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

from webdriver_manager.chrome import ChromeDriverManager

In [3]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/shelbyjouppi/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache
/var/folders/_4/_njb58q944xdbv6xcbzbzmqh0000gn/T/ipykernel_8217/2160715986.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [2]:
headers = requests.utils.default_headers()
headers.update({
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
})

In [3]:
raw_html = requests.get("https://www.psychologytoday.com/us/therapists/javier-moreira-new-york-ny/719894",headers=headers).content
doc = BeautifulSoup(raw_html, "html.parser")
text = doc.get_text()

In [4]:
doc.prettify()

'<!DOCTYPE html>\n<html lang="en-US">\n <head>\n  <meta charset="utf-8"/>\n  <meta content="width=device-width,initial-scale=1.0,maximum-scale=1" name="viewport"/>\n  <meta content="telephone=no" name="format-detection"/>\n  <meta content="Javier Moreira, Counselor, New York, NY, 10016, (646) 760-5852, Javier\'s approach to therapy is primarily person-centered and cognitive-behavioral. However, he also utilizes techniques from other modalities as well including DBT, mindfulness, Gestalt therapy, and motivational interviewing. His therapy involves actively listening to clients\' needs and concerns, while engaging them in whichever therapeutic intervention may best be suitable for them in a welcoming and safe space. Establishing a strong and genuine rapport is essential to help foster change and progress. Javier has worked for over 5 years at an outpatient mental health clinic in Brooklyn, NY." name="description"/>\n  <meta content="index,follow" name="robots"/>\n  <meta content="2419199

In [ ]:
all_profiles = []
for url in df.url:
    raw_html = requests.get(url,headers=headers).content
    doc = BeautifulSoup(raw_html, "html.parser")
    
    profile = {}
    
    profile['url'] = url
    profile['finances'] = doc.find(id="tabs-finances-office").text
    
    all_profiles.append(profile)

In [60]:
finances = doc.find(id="tabs-finances-office").text
# cost = re.findall(r"Cost per Session:.*", finances.text)
# insurance = re.findall(r"Accepted Insurance Plans.*", finances.text)

In [61]:
finances

'\n\n\n\nCost per Session: Up to $200\n            \n\n\n\n\nAccepted Insurance Plans\n\n\n\n                                                            Aetna\n                                                    \n\n                                                            Anthem\n                                                    \n\n                                                            Beacon Health Options\n                                                    \n\n                                                            Cigna and Evernorth\n                                                    \n\n                                                            EmblemHealth\n                                                    \n\n                                                            GEHA\n                                                    \n\n                                                            GHI\n                                                    \n\n           

In [52]:
doc.find(text="Cost per Session:.*")

In [9]:
rows = doc.find_all(class_="results-row")

In [10]:
pt_entries = []
for row in rows:
    data = {}
    
    link = row.find_all('a')[1]
    profile_link = link['href']
    name = link.get_text()
    title = row.find_all(class_='profile-subtitle')[0].get_text()
    phone = row.find_all(class_='results-row-mob')[0].get_text()
    address = row.find_all(class_='profile-location')[0].get_text()
    statement = row.find_all(class_='statements')[0].get_text()
    
    data['url'] = profile_link
    data['name'] = name
    data['title'] = title
    data['phone'] = phone
    data['address'] = address
    data['statement'] = statement
    
    pt_entries.append(data) 

In [28]:
df.url.str.split("?").str[0]

0     https://www.psychologytoday.com/us/therapists/...
1     https://www.psychologytoday.com/us/therapists/...
2     https://www.psychologytoday.com/us/therapists/...
3     https://www.psychologytoday.com/us/therapists/...
4     https://www.psychologytoday.com/us/therapists/...
5     https://www.psychologytoday.com/us/therapists/...
6     https://www.psychologytoday.com/us/therapists/...
7     https://www.psychologytoday.com/us/therapists/...
8     https://www.psychologytoday.com/us/therapists/...
9     https://www.psychologytoday.com/us/therapists/...
10    https://www.psychologytoday.com/us/therapists/...
11    https://www.psychologytoday.com/us/therapists/...
12    https://www.psychologytoday.com/us/therapists/...
13    https://www.psychologytoday.com/us/therapists/...
14    https://www.psychologytoday.com/us/therapists/...
15    https://www.psychologytoday.com/us/therapists/...
16    https://www.psychologytoday.com/us/therapists/...
17    https://www.psychologytoday.com/us/therapi

In [11]:
df = pd.DataFrame(pt_entries)

In [12]:
df.name = df.name.str.replace("\n", "")

In [13]:
df.title = df.title.str.replace("\n", "")

In [14]:
df.title = df.title.astype('str')

In [18]:
def verified(entry):
    query = re.findall(r"Verified",entry)
    if len(query) > 0:
        return "Verified"

In [19]:
df['verified'] = df.title.apply(verified)

In [20]:
df.title = df.title.str.replace("Verified", "")

In [21]:
df

,url,name,title,phone,address,statement,verified
0,https://www.psychologytoday.com/us/therapists/...,Javier Moreira,"Counselor, LMHC",(646) 760-5852,"New York, NY 10016",Javier's approach to therapy is primarily pers...,Verified
1,https://www.psychologytoday.com/us/therapists/...,Heidi Andrea Boyson,"Counselor, LMHC , MS, MA",(718) 865-3041,"New York, NY 10024","I am passionate about providing high quality, ...",Verified
2,https://www.psychologytoday.com/us/therapists/...,Marc A Gerber,"Psychologist, PsyD",(929) 209-5934,"New York, NY 10128",I am a cognitive behavioral therapist with exp...,Verified
3,https://www.psychologytoday.com/us/therapists/...,Brent Bounds,"Psychologist, PhD",(646) 350-0560,"New York, NY 10003",I work collaboratively with my clients to crea...,Verified
4,https://www.psychologytoday.com/us/therapists/...,Nikimya Y Ligon,"Psychologist, MAT, MSEd, CAS, PsyD",(914) 368-7812,"New York, NY 10001",Today you are one step closer to a new you whe...,Verified
5,https://www.psychologytoday.com/us/therapists/...,Kate McConnell,"Marriage & Family Therapist, MA, MFT",(646) 798-4317,"New York, NY 10018",I work primarily with individuals and couples ...,Verified
6,https://www.psychologytoday.com/us/therapists/...,Mitchell S. Levy,"Psychologist, PhD",(646) 381-2963,"New York, NY 10028",Anxiety and depression over this worldwide cri...,Verified
7,https://www.psychologytoday.com/us/therapists/...,Michael Dean Kaltenbach,"Clinical Social Work/Therapist, DSW, LCSW",(323) 510-3581,"New York, NY 10019","Oftentimes, people just need someone to listen...",Verified
8,https://www.psychologytoday.com/us/therapists/...,Brenda Cheryl Kaplan,"Psychologist, PhD",(646) 351-1784,"New York, NY 10025",I am a licensed psychologist with a background...,Verified
9,https://www.psychologytoday.com/us/therapists/...,BASK Consortium,"Clinical Social Work/Therapist, MSW, LCSW-R,...",(914) 246-5818,"New York, NY 10128",Mrs. Angela Keller envisions you being able to...,Verified
